In [20]:
import warnings

import geopandas as gpd
import pandas as pd
import xz

In [34]:
YEAR = 2022

In [35]:
with xz.open(f"../data/statistics/spain/census_tracts_{YEAR}.gpkg.xz") as fp:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", ".*has GPKG application_id.*", RuntimeWarning)
        data = gpd.read_file(fp)

In [36]:
gini = pd.read_csv(
    "../data/statistics/spain/gini_p80p20_tract.zip",
    dtype={"tract_code": str},
)
income = pd.read_csv(
    "../data/statistics/spain/income_tract.zip",
    dtype={"tract_code": str},
)

In [37]:
data.drop(columns="year").merge(gini.query(f"year=={YEAR}"), on="tract_code").query(
    "mun_name == 'Madrid'"
).to_file(f"../data/statistics/madrid/{YEAR}/gini.geojson")

In [38]:
data.drop(columns="year").merge(income.query(f"year=={YEAR}"), on="tract_code").query(
    "mun_name == 'Madrid'"
).to_file(f"../data/statistics/madrid/{YEAR}/income.geojson")

In [39]:
data.drop(columns="year").merge(gini.query(f"year=={YEAR}"), on="tract_code").merge(
    income.query(f"year=={YEAR}").drop(
        ["mun_code", "mun_name", "prov_code", "prov_name", "district_code"], axis=1
    ),
    on="tract_code",
).query("mun_name == 'Madrid'").to_file(
    f"../data/statistics/madrid/{YEAR}/statisitcs.geojson"
)